# 📚 LangChain ile Retrieval Augmented Generation'a Giriş 🦜🔗

Bu not defterinde LangChain kullanarak Retrieval Augmented Generation'ı nasıl kullanacağınızı öğreneceksiniz.

Kendi belgelerimiz hakkında sorular sormak için bir LLM kullanacağız!

## ⚙️ Kurulum

👉 Temel kütüphaneleri içe aktarmak için aşağıdaki hücreyi çalıştırın.

In [ ]:
%load_ext autoreload
%autoreload 2
import os
from pprint import pprint
from IPython.display import Markdown

👉 API anahtarımızı tekrar yüklemek için aşağıdaki hücreyi çalıştırın:

In [ ]:
from dotenv import load_dotenv

load_dotenv()  # Load environment variables from .env file

## 📚 Neden RAG?

Bir LLM kendi başına, öğrendiği her şey hakkında sorulara yanıt verebilir.

Bunun birkaç dezavantajı vardır:
- Eğitim verileri geçmişten gelir ve en son verilerle güncellenmez.
- Sadece eğitim aldığı verileri bilir.

Bir LLM'yi kendi verilerimizle çalışması için kullanmak istiyoruz. İşte bu noktada RAG (Retrieval-Augmented Generation) devreye girer.

1. **Retrieval-Augmented Generation (RAG)**, gerçek doğruluğu artırmak için bir dil modelini belge alıcı ile birleştirir.
2. **İlgili dış belgeleri alır** (örneğin, bilgi tabanından) yanıtlar üretmeden önce.
3. **Dil modeli hem istemi hem de alınan bağlamı kullanarak** daha bilgili ve temelli çıktılar üretir.

## 🇪🇺 Bağlam

Bu meydan okumada, Avrupa Parlamentosu'ndan belgelerle çalışacağız.

Bir gazeteci olduğunuzu ve Avrupa Parlamentosu'nun genel kurul oturumları sırasında belirli bir konu hakkında neler söylendiğini öğrenmek istediğinizi düşünün. Bu oturumlar yılda 12 kez Strasbourg'da gerçekleşir ve 4 gün sürer. Oturumların transkriptleri EP'nin web sitesinde mevcuttur.

Kesinlikle tüm bu transkriptleri karıştırmak istemezsiniz. O halde, hayatımızı kolaylaştırmak için RAG'ı kullanalım!

Bu, her zaman test etmek için yepyeni veriler alabileceğimiz için çalışmak üzere iyi verilerdir.

## 📘 Verileri alalım

1. [EP'nin web sitesine](https://www.europarl.europa.eu/plenary/en/debates-video.html) gidin. 
1. Bu sizi en son genel kurul oturumuna yönlendirecektir.
1. İlk tarihin altında, "▶️ Verbatim reports HTML" bölümünde `HTML`'e tıklayın.
1. Sayfanın sonuna kaydırın ve alttaki PDF dosyasını indirin.
1. Dosyayı `data` klasörüne kaydedin.

Bir belgeyle başlayacağız, ancak daha sonrası için diğer birkaç günün aynısını şimdiden indirebilirsiniz.

Belgeye bir göz atın. Kaç sayfası var? Belge hakkında bir fikir edinmek için hızlıca belgede gezinin.

## 🔢 Belgeleri gömme

Belgeleri gömmek, tüm belgeleri veya belge parçalarını vektörlere çevirmek anlamına gelir.

LangChain🦜🔗 yine çok yardımcı olacak.

Bir gömme aracı (embedder) başlatalım ve deneyelim. LLM olarak Gemini kullandığımız için, Google'ın metin gömme araçlarında kalalım.

In [ ]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

embeddings = GoogleGenerativeAIEmbeddings(model="models/gemini-embedding-001")

👉 Basit bir metin parçasını gömmek için gömme aracının `.embed_query()` metodunu deneyin.

In [ ]:
# Embed a text like "What is the capital of France?" and save it to a variable `sample_embedding`

# YOUR CODE HERE


👉 Bu `sample_embedding`'i keşfetmek için zaman ayırın. Nasıl görünüyor? Tipi nedir? Gömme boyutu nedir?

In [ ]:
# YOUR CODE HERE

## 💾 PDF'den gerçek verilerimizi yükle

Artık bir gömmenin nasıl göründüğünü biliyoruz, gerçek verilerimizle çalışmanın zamanı geldi.

👉 [LangChain belgelerine](https://docs.langchain.com/oss/python/integrations/document_loaders/index#pdfs) gidin ve PyPDF kullanarak bir PDF'yi nasıl yükleyebileceğinizi öğrenin.

👉 Sonra devam edin ve daha önce indirdiğiniz PDF'lerden birini yükleyin.

In [ ]:
# YOUR CODE HERE

👉 `pages`'i keşfedin:
- Veri tipi nedir?
- Kaç sayfanız var?
- Bir sayfanın tipi nedir?
- Bir sayfanın içeriğine nasıl erişebilirsiniz?
- Tam belgenin kaç karakteri var?
- Bir sayfanın `metadata`'sında neler var?

In [ ]:
# YOUR CODE HERE

## ✂️ Verilerimizi böl

Tam belgemiz gömülmek için çok uzun. Metin gömme aracımız 2.048 tokena kadar giriş alabilir. Gemini modelleri için bu yaklaşık 8.196 karakterdir (token başına 4 karakter).

Bu yüzden belgemizi daha küçük parçalara bölmek istiyoruz.

Zaten çalışabileceğimiz bir dizi sayfamız var. Ama sayfa sonları biraz keyfi: genellikle cümlenin ortasında görünürler.

Ayrıca, sayfalar arasında örtüşme yoktur. Bu yüzden bir sayfanın ilk satırı önceki tüm bağlamı kaçırır. Tam metni biraz örtüşmeyle bölmek daha iyidir.

İlk olarak, PDF'yi tekrar yükleyeceğiz, bu sefer bölmeden.

In [ ]:
loader = PyPDFLoader(file_path, mode='single')
pdf = loader.load()
pdf_text = pdf[0].page_content
len(pdf_text)

Artık tüm PDF'imizi tek bir belge olarak aldığımıza göre, onu daha akıllı bir şekilde parçalara bölebiliriz.

👉 Yine, ["Özyinelemeli olarak bölme" konusundaki LangChain belgelerine](https://docs.langchain.com/oss/python/integrations/splitters/recursive_text_splitter) gidin ve `pdf` _belgelerimizi_ parçalara (LangChain'de `documents` olarak adlandırılır) nasıl böleceğinizi öğrenin.

2_000 karakter (bizim durumumuzda yaklaşık yarım sayfa) parçalara 400 örtüşmeyle bölün. İsterseniz diğer değerlerle deneyebilirsiniz.

`RecursiveCharacterTextSplitter`'ın `.split_documents()` metodunu kullanın: bu metod giriş olarak bir belge alır ve bölünmüş belgeler çıktısı verir.

In [ ]:
# YOUR CODE HERE

👉 `all_splits`'i inceleyin:
- Veri tipi nedir?
- Kaç bölümünüz var?
- Bir bölümün tipi nedir?
- Bir bölümün içeriğine nasıl erişebilirsiniz?
- Şimdi toplamda kaç karakterimiz var?
- Bir bölümün `metadata`'sında neler var?

In [ ]:
# YOUR CODE HERE

## 🗄️ Her şeyi bir araya getir: belgelerimizi gömme ve vektör deposunda sakla

Elimizde şunlar var:
- Bir gömme aracı
- Veriyi yüklemek için bir yükleyici
- Belgemizi belgelere bölmek için bir metin bölücü

Neyi kaçırıyoruz?

Belgelerimizi gömebiliriz, ama onları bir yerde saklamak istiyoruz. İşte burada vektör deposu devreye girer: şunları saklamamıza olanak sağlar:
- belgeyi (parçayı),
- onun gömmesini,
- meta verilerini.

Sonraki adımda belgeleri verimli bir şekilde alabilecek olacağız.

👉 Bir `InMemoryVectorStore` nasıl oluşturabileceğinizi görmek için ["Vektör depoları" üzerine LangChain belgelerini](https://docs.langchain.com/oss/python/langchain/knowledge-base#3-vector-stores) kontrol edin.

In [ ]:
# Import the necessary libraries

# YOUR CODE HERE

# Create an in-memory vector store using the embedder `embeddings` we created earlier

# YOUR CODE HERE

# Add the `all_splits` to the vector store and store the result in a variable called `document_ids`

# YOUR CODE HERE


In [ ]:
# Have a look at the first 3 document IDs

# YOUR CODE HERE


In [ ]:
# Use the vector store's `get_by_ids` method. You have to give it a list of document IDs.

# YOUR CODE HERE


👉 Bir vektör deposundaki belgenin içeriğine ve meta verilerine nasıl erişebilirsiniz?

In [ ]:
# YOUR CODE HERE

## 🔎 Benzer belgeleri almak için vektör deposunu kullan

Artık belgeleri gömleğe çevirdiğimize göre, benzer belgeleri almak için vektör deposunu kullanabiliriz.

👉 Bunun nasıl çalıştığını görmek için ["Vektör depoları" üzerine LangChain belgelerini](https://docs.langchain.com/oss/python/langchain/knowledge-base#3-vector-stores) kontrol edin.

Bir sorgu kullanın, örneğin "Tarım politikası üzerine tartışmayı özetle.", ve en benzer belgeleri bulun. Ayrıca alınacak belge sayısını da belirtebilirsiniz.

In [ ]:
# Save your question into a variable called `query`

# YOUR CODE HERE

# Use the vector store to find similar documents to the query. Store the result in a variable called `retrieved_docs`

# YOUR CODE HERE


Bu, RAG'ın sözde "Alma" (Retrieval) kısmını tamamlar: artık sorgumuza en benzer belgeleri bulabiliriz.

Çalışmanın çoğu artık tamamlandı!

## 💬 Sorumuza bir cevap üret

Şimdiye kadar benzer belgeleri almamızı sağlamak için sadece bir **gömme modeli** kullandık.

Şimdi, sorumuzla bir cevap almak için üretici bir LLM kullanacağız: ona aldığımız belgeler ve sorumuzla besleyeceğiz.

Bunu yapmanın en temel yolu tüm girdilerimizi birbirine bağlamak, sorumuzla eklemek ve sonucu görmek olacaktır.

Bir deneyelim.

👉 İlk olarak önceki meydan okumalarda olduğu gibi bir LLM başlatın.

In [ ]:
# YOUR CODE HERE

Sonra temel bir istem oluşturun:

In [ ]:
prompt = '\n\n'.join([doc.page_content for doc in retrieved_docs])
prompt += "\n\n" + query

👉 Şimdi istemi kullanın:

In [ ]:
# YOUR CODE HERE

Bu fena değil, ama modele daha fazla rehberlik vererek daha kapsamlı bir istem yazarak daha iyisini yapabiliriz.

Bunu yapan ilk kişiler biz değilmişiz ve LangChain'in bizim için önceden hazırlanmış istem kütüphanesi var.

👉 Aşağıdaki hücreyi çalıştırın ve nasıl çalıştığını anlamaya çalışın. (LangSmithMissingAPIKeyWarning hakkında bir uyarı alacaksınız, bunu görmezden gelebilirsiniz.)

In [ ]:
from langchain_classic import hub

prompt_template = hub.pull("rlm/rag-prompt")

example_messages = prompt_template.invoke(
    {"context": "(context goes here)", "question": "(question goes here)"}
).to_messages()

print("\n")
print(example_messages[0].content)

LangChain'in bizim için nasıl daha kesin bir istem oluşturduğunu görüyor musunuz? Bunu RAG'ımız için kullanalım!

👉 İlk olarak, tüm alınan belgeleri iki yeni satırla ayrılmış tek bir uzun dizgiye birleştirin.

In [ ]:
# YOUR CODE HERE

👉 Sonra, sorgunuz ve alınan belgelerden başlayarak bir `prompt` oluşturun. Yukarıdaki örneğe bakmayı unutmayın.

In [ ]:
# YOUR CODE HERE

👉 Son olarak az önce oluşturduğumuz `the_prompt` ile LLM modelini kullanın:

In [ ]:
# YOUR CODE HERE

🎉 İlk RAG'ımızı tamamladık: LLM kendisine sağladığımız belgelerde ***temelli*** metin üretti.

## 💾 Gömmelerimizi kalıcı hale getir

Şimdiye kadar bellekte vektör deposuyla çalıştık. Bu yüzden not defterinizi kapattığınızda, tüm gömmeleri de kaybedeceksiniz.

⚠️ Bu gömmelerin sağlayıcınızın platformunda, bu durumda Google'ın makinelerinde çalışan modeller tarafından üretildiğini unutmayın. Ve bedava çalışmazlar. 💰

Bunun gibi bir, nispeten küçük belge için maliyet düşüktür, ama hızla artar. Şimdiye kadar sadece bir günün transkriptleriyle çalıştık. Oturum başına 3 tane daha, yılda 12 oturum, birden fazla yıl var...

Bunu çözmek için sadece vektör depomuzla kalıcı bir taneyi değiştireceğiz. Bu LangChain'in avantajıdır: bileşenleri değiştirmek çok kolay.

Bellekteki vektör depomuz deneme için harikaydı, şimdi başka bir taneyle değiştireceğiz. Çok popüler bir vektör deposu olan [Chroma](https://www.trychroma.com/)'yı kullanacağız. Bunu yerel olarak çalıştırabilir ve LangChain aracılığıyla kullanabiliriz.

Tüm akışımızı yeniden oluşturacağız. Her şeyi birkaç kod hücresinde tekrar bir araya getirmeye çalışmak iyi bir alıştırmadır. Aynı zamanda her şeyi yeniden kullanılabilir koda dönüştüreceğiz.

Sonunda iki fonksiyon istiyoruz:

1. `embed_and_store()`: Başka bir oturumun transkriptini vektör veritabanımıza ekle, böylece alacağımız daha fazla veri olsun.
2. `answer()`: Vektör depomuzla farklı sorularla sorgula.

#### 1. Bir Chroma vektör deposu başlat

👉 **Veri kalıcılığıyla** (yani verileri diskteki bir dizinde saklayarak) Chroma vektör deposunun nasıl oluşturulacağını görmek için [LangChain'in belgelerine](https://python.langchain.com/docs/integrations/vectorstores/chroma/) bakın.

In [ ]:
# YOUR CODE HERE

#### 2. `embed_and_store()` oluştur

👉 Bu fonksiyon için kodu tamamlayın:

In [ ]:
def embed_and_store(file_path, vector_store):
    """Load a PDF file, split it into chunks, and store the chunks in a vector store."""
    # Load the PDF file
    pass  # YOUR CODE HERE

    # Split the pages into chunks
    pass  # YOUR CODE HERE

    # # Add the session_date to the metadata
    # for split in all_splits:
    #     split.metadata['session_date'] = session_date

    # Add the chunks to the vector store
    pass  # YOUR CODE HERE

    return document_ids

👉 Fonksiyonunuzu bir dosya veya hatta iki dosyayla deneyin:

In [ ]:
# YOUR CODE HERE

#### 3. `answer()` oluştur

👉 Bu fonksiyon için kodu tamamlayın:

In [ ]:
def answer(query, vector_store, llm, prompt_template=None):
    """Answer a query using the vector store and the language model."""
    # Retrieve similar documents from the vector store
    retrieved_docs = vector_store.similarity_search(query, k=6)

    # Create the prompt
    docs_content = "\n\n".join(doc.page_content for doc in retrieved_docs)

    # If no prompt template is provided, use the default one
    if not prompt_template:
        prompt_template = hub.pull("rlm/rag-prompt")

    prompt = prompt_template.invoke(
        {"context": docs_content, "question": query}
    )

    # Get the answer from the language model
    answer = llm.invoke(prompt)

    return answer.content

👉 Fonksiyonunuzu beğendiğiniz bir sorguyla deneyin:

In [ ]:
# YOUR CODE HERE

🏁 Tebrikler! Artık LangChain kullanarak RAG'da ustalaştınız ve vektör deponuza daha fazla belge eklemek ve onu sorgulamak için yeniden kullanılabilir fonksiyonlar yapmayı öğrendiniz.

## [İsteğe Bağlı] Meta veri ekleme

Kurduğumuz RAG, vektör deposundaki tüm belgeleri sorgular. Orada birden fazla yılın bilgisinin olduğunu düşünün. Yıllara veya tarihlere göre filtreyebilsek kullanışlı olurdu, değil mi?

Bunu nasıl yaparız? Vektör deposundaki belgelerin meta veri içerdiğini unutmayın. Eğer tarihi ekleyebilseydik, daha sonra filtrelemek için kullanabilirdik.

İpucu: Meta verilerinizi pipeline'ınızda olabildiğince erken ekleyin. Verileriniz vektör deposunda saklandıktan sonra eklemeye çalışmayın.

👉 `embed_and_store()` fonksiyonunuzu uyarlayın.

In [ ]:
def embed_and_store_fancy(file_path, vector_store, session_date):
    """Load a PDF file, split it into chunks, and store the chunks in a vector store.
    Session_date is added to the metadata of each chunk."""
    pass  # YOUR CODE HERE

    return document_ids

👉 Fonksiyonunuzu deneyin ve vektör deponuzun ek meta veri içerdiğini kontrol edin.

In [ ]:
# YOUR CODE HERE

Şimdi alıcıyı kullanıcının sorduğu tarihe göre sınırlamamız gerekiyor.

👉 `answer()` fonksiyonunuzu bir tarih alabilecek ve yeni meta verilere dayalı olarak belgeleri filtreleyebilecek şekilde uyarlayın.

In [ ]:
# YOUR CODE HERE

In [ ]:
# YOUR CODE HERE

Harika! Güçlü bir RAG sistemi oluşturmak için benzerlik aramasını meta veri aramasıyla birleştirdiniz!